In [1]:
import pandas as pd
import numpy as np
import re
import os
import json
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 1000+1)

In [2]:
pdb_out_dir = 'dataset/Godess_gnn/preprocess_data/Godess_carbon_unmatched_pdb/'

csv_out_dir = 'dataset/Godess_gnn/preprocess_data/Godess_carbon_unmatched_carbon_csv/'
csv_out_dir_hydrogen = 'dataset/Godess_gnn/preprocess_data/Godess_carbon_unmatched_hydrogen_csv/'


matching_out_dir = 'dataset/Godess_gnn/preprocess_data/Godess_carbon_match_dict/'
connection_out_dir = 'dataset/Godess_gnn/preprocess_data/Godess_carbon_connection_dict/'

pdb_file_list = os.listdir(pdb_out_dir)

partial_labeled_dir = 'dataset/Godess_gnn/preprocess_data/Godess_carbon_labeled_pdb_monosaccharide/'
all_labeled_dir = 'dataset/Godess_gnn/preprocess_data/Godess_carbon_labeled_pdb_all/'

In [3]:
def match_residual_accurate_name(pdb_f, csv_f):
    for i in range(len(pdb_f)):
        current_pdb_lineage = pdb_f.loc[i, :]['Lineage']
        if current_pdb_lineage in csv_f['Linkage'].values:

            current_pdb_residue_list = csv_f.loc[csv_f['Linkage'] == current_pdb_lineage]['Residue'].values
            
            if ('Ac' in current_pdb_residue_list) and (len(current_pdb_residue_list) >= 2):
                current_ac_index = np.where(current_pdb_residue_list == 'Ac')[0]
                current_pdb_residue_list = np.delete(current_pdb_residue_list, current_ac_index)
            
            assert (len(current_pdb_residue_list) == 1)
            
            current_corresponding_residue = current_pdb_residue_list[0]
        #         print(current_corresponding_residue)
            pdb_f.loc[i, ['Residual_accurate_name']] = current_corresponding_residue
    return pdb_f

##### Additional parts

In [4]:
# The connection information from the pdb file is cannot be fully trusted, some file have the patterns that A -> B, but B is not connected to A
def check_and_modify_graph(graph):
    """
    Checks if a graph satisfies the property that if node A is connected to node B, then node B is connected to node A.
    If the property is not satisfied, modifies the graph by adding the missing connections.
    
    Arguments:
    graph -- a dictionary representing a graph where the keys are the nodes and the values are lists of connected nodes
    
    Returns:
    The modified graph.
    """
    for node in graph:
        for neighbor in graph[node]:
            if node not in graph[neighbor]:
                graph[neighbor].append(node)
    return graph

In [5]:
# read in connection from step 2 (create linkage for pdb file)
def read_in_connection(connection_dir, pdb_name):
    
#     name = pdb_name.split('labeled_all_')[1].split('.pdb')[0]
    name = pdb_name.split('.pdb.csv')[0]
    connect_name = str(name) + '_connection.json'
    
    dict_f_path = os.path.join(connection_dir, connect_name)
    
    with open(dict_f_path) as dict_f_json:
        dict_f = json.load(dict_f_json)
    
    dict_f = {int(k):v for k,v in dict_f.items()}
    
    dict_f = check_and_modify_graph(dict_f)
    
    return dict_f

In [6]:
# given target root, for each atom, find the shortest distance to the root atom (Carbon) using BFS
def bfs_shortest_path(graph, start_node, end_node):
    """
    Compute the shortest path from start_node to end_node in graph using Breadth-First Search.
    
    Arguments:
    graph -- a dictionary representing the graph, where keys are nodes and values are lists of neighbors
    start_node -- the node to start the search from
    end_node -- the node to search for
    
    Returns:
    A list containing the shortest path as a list of nodes.
    """
    # Initialize the visited set, the queue, and the parent dictionary with start_node as the only node visited.
    visited = set([start_node])
    queue = deque([(start_node, [])])
    parent = {start_node: None}
    
    while queue:
        # Dequeue the next node and its path.
        current_node, path = queue.popleft()
        
        # If we've found the end_node, return its path.
        if current_node == end_node:
            return path + [end_node]
        
        # Loop through the neighbors of the current_node and enqueue them if they haven't been visited.
        for neighbor in graph[current_node]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, path + [current_node]))
                parent[neighbor] = current_node
    
    # If we've gone through all the nodes and haven't found the end_node, return an empty path.
    return []

In [7]:
# identify Ac components with the help of connection information, branched info from labedled csv and pdb residue names
def identify_AC_components(df_pdb, df_pdb_name, df_label, connection_dir, j):

    dict_f = read_in_connection(connection_dir, df_pdb_name)

    df_pdb_copy = df_pdb.copy()

    df_label = df_label

    Ac_components_list = np.repeat(0, len(df_pdb_copy))

    for i in range(len(df_label)):
        current_linkage = df_label.loc[i, :]['Linkage']

        if isinstance(current_linkage, float):
            current_linkage = int(current_linkage)
            current_linkage = str(current_linkage)

        current_residue = df_label.loc[i, :]['Residue']

        if current_residue == 'Ac':

            attached_linkage = current_linkage[0:-2]

            attached_index = df_pdb_copy.loc[df_pdb_copy['Lineage'] == attached_linkage].index

            attached_atom_name = df_pdb_copy.loc[df_pdb_copy['Lineage'] == attached_linkage, :]['Atom_name'].values

            if 'C11' in attached_atom_name:
                c11_index_list = np.where(attached_atom_name == 'C11')[0]

                # Attached only one monosaccharide
                if len(c11_index_list) == 1:

                    # c11 index in csv
                    c11_index = attached_index[c11_index_list[0]]

                    # c11 index in atom names
                    c11_atom_num = df_pdb_copy.loc[c11_index, :]['Atom_num']

                    c11_connection_list = dict_f[c11_atom_num]

                    Ac_residue_index_list = [c11_atom_num]

                    for atom_num in c11_connection_list:

                        current_atom_connect = dict_f[atom_num]
                        
                        # in case of the connection file failed
                        if c11_atom_num in current_atom_connect:
                            current_atom_connect.remove(c11_atom_num)

                        # identify the Oxx
                        if len(current_atom_connect) == 0:

                            Ac_residue_index_list.append(atom_num)

                        else:
                            len_atom_connect = 0
                            for atom_num_connect in current_atom_connect:


                                atom_num_connect_atom_list = dict_f[atom_num_connect]
                                
                                if atom_num in atom_num_connect_atom_list:

                                    atom_num_connect_atom_list.remove(atom_num)
    #                             print(atom_num_connect, atom_num_connect_atom_list, atom_num)

                                len_atom_connect += len(atom_num_connect_atom_list)

                            # identify the C21, and three HXXs 
                            if len_atom_connect == 0:

                                Ac_residue_index_list.append(atom_num)
                                Ac_residue_index_list.extend(dict_f[atom_num])

                    Ac_residue_index_list = df_pdb_copy.loc[df_pdb_copy['Atom_num'].isin(Ac_residue_index_list)].index
                    Ac_components_list[Ac_residue_index_list] = 1
    #                     print(atom_num, current_atom_connect)
                else:
                    print(df_pdb_name, j, 'contain more than one C11')


            else:

                print(df_pdb_name, j, current_residue, 'the attached monosaccharides does not contain a C11')

                current_index = df_pdb_copy.loc[df_pdb_copy['Lineage'] == current_linkage].index

                current_three_letter_residual = df_pdb_copy.loc[current_index, :]['Residual_name'].values

                # the attached monosaccharide 
                assert(np.all(current_three_letter_residual == 'ACY'))

    # some ACs are behave like are separate components
    if not df_pdb_copy.loc[df_pdb_copy['Residual_name'] == 'ACY'].empty:
        ACY_index = df_pdb_copy.loc[df_pdb_copy['Residual_name'] == 'ACY'].index
        Ac_components_list[ACY_index] = 1

    df_pdb_copy['Ac_component'] = Ac_components_list
    
    return df_pdb_copy

##### label the shift for monosaccharides

In [8]:
def match_pdb_csv_lineage(pdb_f, csv_f, carbon_list, ac_carbon_list = ['C11', 'C21'], trust_threshold = 50):
    
    csv_f['Linkage'] = csv_f[['Linkage']].fillna('')
    csv_f = csv_f.astype({'Linkage':str})
    if ('1.0' in csv_f['Linkage'].values) or ('2.0' in csv_f['Linkage'].values) or ('3.0' in csv_f['Linkage'].values) or \
        ('4.0' in csv_f['Linkage'].values) or ('5.0' in csv_f['Linkage'].values) or ('6.0' in csv_f['Linkage'].values):
        print('error in pdb lineage')
    
    for i in range(len(pdb_f)):
        current_pdb_lineage = pdb_f.loc[i, :]['Lineage']
        current_pdb_atom = pdb_f.loc[i, :]['Atom_name']
        current_pdb_residual_name = pdb_f.loc[i, :]['Residual_name'] 

        # stores the monosaccharide labels 
        current_corresponding_csv_f = csv_f.loc[csv_f['Linkage'] == current_pdb_lineage]

        # stores the Ac labels
        current_corresponding_csv_f_ac = pd.DataFrame(columns=csv_f.columns)

        if 'Ac' in current_corresponding_csv_f['Residue'].values:

            current_corresponding_csv_f_ac = current_corresponding_csv_f.\
                loc[current_corresponding_csv_f['Residue'] == 'Ac']

            current_corresponding_csv_f = current_corresponding_csv_f.\
                loc[current_corresponding_csv_f['Residue'] != 'Ac']

        # current monosaccharide, assign shift
        if (current_pdb_atom in carbon_list) and (not current_corresponding_csv_f.empty):

            current_shift = current_corresponding_csv_f[current_pdb_atom].values[0]
            
            if (len(current_corresponding_csv_f['Trust'].values) == 0) or (current_corresponding_csv_f['Trust'].values ==''):
                print(current_pdb_residual_name)
                print('Missing trust confidence interval: ', current_corresponding_csv_f['Residue'])
                current_trust = -1

            elif isinstance(current_corresponding_csv_f['Trust'].values[0], float) and \
                np.isnan(current_corresponding_csv_f['Trust'].values[0]):

                print('Missing trust confidence interval: ', current_corresponding_csv_f['Residue'])
                current_trust = -1
            else:

                current_trust = current_corresponding_csv_f['Trust'].values[0]
                current_trust = int(current_trust.split('%')[0])

            pdb_f.loc[i, ['shift']] = current_shift
            pdb_f.loc[i, ['trust']] = current_trust

        # current Ac and the pdb file has a separate three letter code
        elif (current_pdb_atom in carbon_list) and (not current_corresponding_csv_f_ac.empty) \
            and (current_pdb_residual_name == 'ACY'):

            current_shift = current_corresponding_csv_f_ac[current_pdb_atom].values[0]

            if (len(current_corresponding_csv_f['Trust'].values) == 0) or (current_corresponding_csv_f['Trust'].values ==''):
                print(current_pdb_residual_name)
                print('Missing trust confidence interval: ', current_corresponding_csv_f['Residue'])
                current_trust = -1

            elif isinstance(current_corresponding_csv_f_ac['Trust'].values[0], float) and \
                np.isnan(current_corresponding_csv_f_ac['Trust'].values[0]):

                print('Missing trust confidence interval: ', current_corresponding_csv_f_ac['Residue'])
                current_trust = -1
            else:
                current_trust = current_corresponding_csv_f_ac['Trust'].values[0]
                current_trust = int(current_trust.split('%')[0])

            pdb_f.loc[i, ['shift']] = current_shift
            pdb_f.loc[i, ['trust']] = current_trust

        elif (current_pdb_atom in ac_carbon_list):
#             print(i)
            pass
        
    return pdb_f

In [9]:
def match_pdb_csv_lineage_hydrogen(pdb_f, csv_hydrogen_f, hydrogen_list, ac_hydrogen_list = ['H2'], 
                                   trust_threshold = 50):
    
    csv_hydrogen_f['Linkage'] = csv_hydrogen_f[['Linkage']].fillna('')
    csv_hydrogen_f = csv_hydrogen_f.astype({'Linkage':str})
    if ('1.0' in csv_hydrogen_f['Linkage'].values) or ('2.0' in csv_hydrogen_f['Linkage'].values) or ('3.0' in csv_hydrogen_f['Linkage'].values) or \
        ('4.0' in csv_hydrogen_f['Linkage'].values) or ('5.0' in csv_hydrogen_f['Linkage'].values) or ('6.0' in csv_hydrogen_f['Linkage'].values):
        print('error in pdb lineage')
    
    for i in range(len(pdb_f)):
        current_pdb_lineage = pdb_f.loc[i, :]['Lineage']
        current_pdb_atom = pdb_f.loc[i, :]['Atom_name']
        current_pdb_residual_name = pdb_f.loc[i, :]['Residual_name'] 

        # stores the monosaccharide labels 
        current_corresponding_csv_hydrogen_f = csv_hydrogen_f.loc[csv_hydrogen_f['Linkage'] == current_pdb_lineage]

        # stores the Ac labels
        current_corresponding_csv_hydrogen_f_ac = pd.DataFrame(columns=csv_hydrogen_f.columns)

        if 'Ac' in current_corresponding_csv_hydrogen_f['Residue'].values:

            current_corresponding_csv_hydrogen_f_ac = current_corresponding_csv_hydrogen_f.\
                loc[current_corresponding_csv_hydrogen_f['Residue'] == 'Ac']

            current_corresponding_csv_hydrogen_f = current_corresponding_csv_hydrogen_f.\
                loc[current_corresponding_csv_hydrogen_f['Residue'] != 'Ac']

        # current monosaccharide, assign shift,
        if (current_pdb_atom in hydrogen_list) and (not current_corresponding_csv_hydrogen_f.empty):

            current_shift = current_corresponding_csv_hydrogen_f[current_pdb_atom].values[0]
            
            if (len(current_corresponding_csv_hydrogen_f['Trust'].values) == 0) or (current_corresponding_csv_hydrogen_f['Trust'].values ==''):
                print(current_pdb_residual_name)
                print('Missing trust confidence interval: ', current_corresponding_csv_hydrogen_f['Residue'])
                current_trust = -1

            elif isinstance(current_corresponding_csv_hydrogen_f['Trust'].values[0], float) and \
                np.isnan(current_corresponding_csv_hydrogen_f['Trust'].values[0]):

                print('Missing trust confidence interval: ', current_corresponding_csv_hydrogen_f['Residue'])
                current_trust = -1
            else:

                current_trust = current_corresponding_csv_hydrogen_f['Trust'].values[0]
                current_trust = int(current_trust.split('%')[0])

            pdb_f.loc[i, ['shift']] = current_shift
            pdb_f.loc[i, ['trust']] = current_trust

        # current Ac and the pdb file has a separate three letter code
        elif (current_pdb_atom in hydrogen_list) and (not current_corresponding_csv_hydrogen_f_ac.empty) \
            and (current_pdb_residual_name == 'ACY'):

            current_shift = current_corresponding_csv_hydrogen_f_ac[current_pdb_atom].values[0]
            
            if (len(current_corresponding_csv_hydrogen_f['Trust'].values) == 0) or (current_corresponding_csv_hydrogen_f['Trust'].values ==''):
                print(current_pdb_residual_name)
                print('Missing trust confidence interval: ', current_corresponding_csv_hydrogen_f['Residue'])
                current_trust = -1

            elif isinstance(current_corresponding_csv_hydrogen_f_ac['Trust'].values[0], float) and \
                np.isnan(current_corresponding_csv_hydrogen_f_ac['Trust'].values[0]):

                print('Missing trust confidence interval: ', current_corresponding_csv_hydrogen_f_ac['Residue'])
                current_trust = -1
            else:
                current_trust = current_corresponding_csv_hydrogen_f_ac['Trust'].values[0]
                current_trust = int(current_trust.split('%')[0])

            pdb_f.loc[i, ['shift']] = current_shift
            pdb_f.loc[i, ['trust']] = current_trust

        elif (current_pdb_atom in ac_hydrogen_list):
#             print(i)
            pass
        
    return pdb_f

In [10]:
def match_Ac_carbon_hydrogen_in_monosaccharide(new_pdb_f, csv_f, csv_hydrogen_f, j):
    pdb_f = new_pdb_f.copy()
    pdb_f_lineage_list = pdb_f['Lineage'].values
    for i in range(len(csv_f)):
        current_csv_lineage = csv_f.loc[i, :]['Linkage']
        current_csv_residue = csv_f.loc[i, :]['Residue']

        if (current_csv_lineage not in pdb_f_lineage_list) and \
            (current_csv_lineage[:-2] in pdb_f_lineage_list):
                current_csv_attached_lineage = current_csv_lineage[:-2]

                temp_ac_pdb_f = pdb_f.loc[(pdb_f['Ac_component'] == 1) & \
                                            (pdb_f['Lineage'] == current_csv_attached_lineage)]
                
                if not temp_ac_pdb_f.empty:
                
                    temp_ac_pdb_f_atoms = temp_ac_pdb_f['Atom_name'].values

                    # assign carbon shift
                    # ['C1', 'C11', 'C2', 'C21', 'H2', 'H21', 'H22', 'H23', 'O1', 'O11','O12']

                    ac_c1_shift_gt = csv_f.loc[i, :]['C1']
                    ac_c2_shift_gt = csv_f.loc[i, :]['C2']
                    ac_c_trust_gt = csv_f.loc[i, :]['Trust']

                    if '%' in ac_c_trust_gt:
                        ac_c_trust_gt = int(ac_c_trust_gt.split('%')[0])
                    else:
                        ac_c_trust_gt = -1


                    if 'C11' in temp_ac_pdb_f_atoms:
                        c11_idx = temp_ac_pdb_f.loc[temp_ac_pdb_f['Atom_name'] == 'C11'].index
                        pdb_f.loc[c11_idx, ['shift']] = ac_c1_shift_gt
                        pdb_f.loc[c11_idx, ['Trust']] = ac_c_trust_gt

                    if 'C1' in temp_ac_pdb_f_atoms:
                        c11_idx = temp_ac_pdb_f.loc[temp_ac_pdb_f['Atom_name'] == 'C1'].index
                        pdb_f.loc[c11_idx, ['shift']] = ac_c1_shift_gt
                        pdb_f.loc[c11_idx, ['Trust']] = ac_c_trust_gt

                    assert( not (('C11' in temp_ac_pdb_f_atoms) and ('C1' in temp_ac_pdb_f_atoms)) )

                    if 'C21' in temp_ac_pdb_f_atoms:
                        c21_idx = temp_ac_pdb_f.loc[temp_ac_pdb_f['Atom_name'] == 'C21'].index
                        pdb_f.loc[c21_idx, ['shift']] = ac_c2_shift_gt

                    if 'C2' in temp_ac_pdb_f_atoms:
                        c21_idx = temp_ac_pdb_f.loc[temp_ac_pdb_f['Atom_name'] == 'C21'].index
                        pdb_f.loc[c21_idx, ['shift']] = ac_c2_shift_gt

                    assert( not (('C21' in temp_ac_pdb_f_atoms) and ('C2' in temp_ac_pdb_f_atoms)) )

                    # assign carbon shift
                    # # ['C1', 'C11', 'C2', 'C21', 'H2', 'H21', 'H22', 'H23', 'O1', 'O11','O12']
                    ac_h1_shift_gt = csv_hydrogen_f.loc[i, :]['H1']
                    ac_h_trust_gt = csv_hydrogen_f.loc[i, :]['Trust']

                    if '%' in ac_h_trust_gt:
                        ac_h_trust_gt = int(ac_h_trust_gt.split('%')[0])
                    else:
                        ac_h_trust_gt = -1

                    if (('H2' in temp_ac_pdb_f_atoms) and ('H21' in temp_ac_pdb_f_atoms) and 
                        ('H22' in temp_ac_pdb_f_atoms)):

                        h2_idx = temp_ac_pdb_f.loc[temp_ac_pdb_f['Atom_name'] == 'H2'].index
                        pdb_f.loc[h2_idx, ['shift']] = ac_h1_shift_gt
                        pdb_f.loc[h2_idx, ['Trust']] = ac_h_trust_gt

                    elif (('H21' in temp_ac_pdb_f_atoms) and ('H22' in temp_ac_pdb_f_atoms) and 
                          ('H23' in temp_ac_pdb_f_atoms)):

                        h2_idx = temp_ac_pdb_f.loc[temp_ac_pdb_f['Atom_name'] == 'H21'].index
                        pdb_f.loc[h2_idx, ['shift']] = ac_h1_shift_gt
                        pdb_f.loc[h2_idx, ['Trust']] = ac_h_trust_gt
                    else:
                        print(j, temp_ac_pdb_f_atoms, 'wierd h2')
    return pdb_f

In [11]:
# def match_Ac_in_monosaccharide(csv_f, pdb_f, ac_carbon_list = ['C11', 'C21'], 
#                                corres_ac_carbon_list = ['C1', 'C2'], exclude_mono_list = ['Pyr']):
    
#     pdb_f_lineage_list = pdb_f['Lineage'].values
#     for i in range(len(csv_f)):
#         current_csv_lineage = csv_f.loc[i, :]['Linkage']
#         current_csv_residue = csv_f.loc[i, :]['Residue']
        
#         current_csv_res = csv_f.loc[i, :]['Residue']
        
#         if (current_csv_lineage not in pdb_f_lineage_list) and (current_csv_res not in exclude_mono_list):
#             current_csv_attached_lineage = current_csv_lineage[:-2]
# #             print(current_csv_lineage, current_csv_attached_lineage)

#             for j in range(len(ac_carbon_list)):
#                 ac_c_shift = csv_f.loc[i, :][corres_ac_carbon_list[j]]
#     #             print(ac_c_shift)
    
#                 ac_c_trust = csv_f.loc[i, :]['Trust']
            
#                 if '%' in ac_c_trust:
#                     ac_c_trust = int(ac_c_trust.split('%')[0])
#                 else:
#                     ac_c_trust = -1
    
#                 pdb_f.loc[(pdb_f['Lineage'] == current_csv_attached_lineage) & 
#                           (pdb_f['Atom_name'] == ac_carbon_list[j]), ['shift']] = ac_c_shift
        
#                 pdb_f.loc[(pdb_f['Lineage'] == current_csv_attached_lineage) & 
#                       (pdb_f['Atom_name'] == ac_carbon_list[j]), ['trust']] = ac_c_trust
#     return pdb_f

In [12]:
# def match_Ac_in_monosaccharide_hydrogen(csv_hydrogen_f, pdb_f, ac_hydrogen_list = ['H2'], 
#                                corres_ac_hydrogen_list = ['H2'], exclude_mono_list = ['Pyr']):
    
#     pdb_f_lineage_list = pdb_f['Lineage'].values
#     for i in range(len(csv_hydrogen_f)):
#         current_csv_lineage = csv_hydrogen_f.loc[i, :]['Linkage']
#         current_csv_residue = csv_hydrogen_f.loc[i, :]['Residue']
        
#         current_csv_res = csv_f.loc[i, :]['Residue']
        
#         if (current_csv_lineage not in pdb_f_lineage_list) and (current_csv_res not in exclude_mono_list):
#             current_csv_attached_lineage = current_csv_lineage[:-2]
# #             print(current_csv_lineage, current_csv_attached_lineage)

#             for j in range(len(ac_hydrogen_list)):
#                 ac_c_shift = csv_hydrogen_f.loc[i, :][corres_ac_hydrogen_list[j]]
#     #             print(ac_c_shift)
    
#                 ac_c_trust = csv_hydrogen_f.loc[i, :]['Trust']
            
#                 if '%' in ac_c_trust:
#                     ac_c_trust = int(ac_c_trust.split('%')[0])
#                 else:
#                     ac_c_trust = -1
    
#                 pdb_f.loc[(pdb_f['Lineage'] == current_csv_attached_lineage) & 
#                           (pdb_f['Atom_name'] == ac_hydrogen_list[j]), ['shift']] = ac_c_shift
        
#                 pdb_f.loc[(pdb_f['Lineage'] == current_csv_attached_lineage) & 
#                       (pdb_f['Atom_name'] == ac_hydrogen_list[j]), ['trust']] = ac_c_trust
#     return pdb_f

In [13]:
# for i in tqdm(range(len(pdb_file_list))):
# #     if i in illegal_list:
# #         continue
    
#     pdb_f_name = pdb_file_list[i]
#     if '1494' in pdb_f_name:
#         print(i, pdb_f_name)

In [14]:
for i in tqdm(range(len(pdb_file_list))):


# for i in [0]:
# for i in [14]:
# for i in [59]:
# for i in [488]:
# for i in tqdm(range(1024, len(pdb_file_list))):

# for i in tqdm(range(2350, len(pdb_file_list))):

# for i in [407]:

#     if i in illegal_list:
#         continue
    
    pdb_f_name = pdb_file_list[i]
    
#     print(i, pdb_f_name)
    
    csv_f_name = pdb_f_name.replace('.pdb.csv', '.csv')
    dict_f_name = pdb_f_name.replace('.pdb.csv', '_matching.json')
    connect_f_name = pdb_f_name.replace('.pdb.csv', '_connection.json')
    
    # read in the dataset
    pdb_f_path = os.path.join(pdb_out_dir, pdb_f_name)
    
    csv_f_path = os.path.join(csv_out_dir, csv_f_name)
    csv_hydrogen_f_path = os.path.join(csv_out_dir_hydrogen, csv_f_name)
    dict_f_path = os.path.join(matching_out_dir, dict_f_name)
    connect_f_path = os.path.join(connection_out_dir, connect_f_name)
    
    pdb_f = pd.read_csv(pdb_f_path)
    
    # carbon 
    csv_f = pd.read_csv(csv_f_path, keep_default_na=False)
    csv_f = csv_f.loc[csv_f['Residue'] != 'PDa']
    csv_f.index = range(len(csv_f))
    
    #hydrogen
    csv_hydrogen_f = pd.read_csv(csv_hydrogen_f_path, keep_default_na=False)
    csv_hydrogen_f = csv_hydrogen_f.loc[csv_hydrogen_f['Residue'] != 'PDa']
    csv_hydrogen_f.index = range(len(csv_hydrogen_f))
    
    
#     lineage_length = len(np.unique(csv_f['Lineage'].values.astype(str)))
    
#     csv_length = len(csv_f)
    
#     assert(lineage_length == csv_length)
    
#     if csv_f.loc[0, ['Residue']].values[0] == 'Ac':
#         print(i)
    
    with open(dict_f_path) as dict_f_json:
        dict_f = json.load(dict_f_json)
    
    with open(connect_f_path) as connect_f_json:
        connect_f = json.load(connect_f_json)
    
    # assign pdb files by lineage from connection dict file
    pdb_f['Residual_accurate_name'] = 'Missing Monosaccharide'
    pdb_f['Lineage'] = 'Missing Lineage'
    pdb_f['shift'] = -1
    pdb_f['trust'] = -1
    
    C1_index = np.where(csv_f.columns == 'C1')[0][0]
    carbon_list = csv_f.columns[C1_index:].values
    
    H1_index = np.where(csv_hydrogen_f.columns == 'H1')[0][0]
    hydrogen_list = csv_hydrogen_f.columns[H1_index:].values
    
    
    for current_dict_residual, current_dict_lineage in sorted(dict_f.items()):
        current_dict_residual = int(current_dict_residual)
        pdb_f.loc[pdb_f['Residual_num'] == current_dict_residual, ['Lineage']] = current_dict_lineage
    
    
    # use assigned lineage, assign pdb files by shift and monosaccharides from label file
    
    if 'ACY' in pdb_f['Residual_name'].values:
        print(i, pdb_f_name)
        pass

#     for j in pdb_f['Atom_name'].values:
#         if len(j) >= 4:
#             print(i, j, pdb_f_name)
#     print(pdb_f_name)
    pdb_f = match_residual_accurate_name(pdb_f, csv_f)
    
    
    # match monosaccharide shift for carbon and hydrogen 
    
    new_pdb_f = match_pdb_csv_lineage(pdb_f, csv_f, carbon_list)
    
    
    new_pdb_f = match_pdb_csv_lineage_hydrogen(new_pdb_f, csv_hydrogen_f, hydrogen_list)
    
    
    
    #############################################################
    ####identify the Ac componenets
    #############################################################
    
    
    new_pdb_f = identify_AC_components(new_pdb_f, pdb_f_name, csv_f, connection_out_dir, i)
    
    
    # match monosaccharide shift for carbon and hydrogen
    
    
    new_pdb_f_with_ac = match_Ac_carbon_hydrogen_in_monosaccharide(new_pdb_f, csv_f, csv_hydrogen_f, i)
#     new_pdb_f_with_ac = match_Ac_in_monosaccharide(csv_f, new_pdb_f)
    
    
#     new_pdb_f_with_ac = match_Ac_in_monosaccharide_hydrogen(csv_hydrogen_f, new_pdb_f_with_ac)
    
    
    current_out_pdb_path = os.path.join(partial_labeled_dir, 'labeled_mono_' + pdb_f_name)
    current_out_pdb_path_complete = os.path.join(all_labeled_dir, 'labeled_all_' + pdb_f_name)
    
    
    new_pdb_f.to_csv(current_out_pdb_path, index = False)
    new_pdb_f_with_ac.to_csv(current_out_pdb_path_complete, index = False) 

  1%|▎                                        | 17/2402 [00:10<18:01,  2.20it/s]

17 209.pdb.csv
ACY
Missing trust confidence interval:  Series([], Name: Residue, dtype: object)
ACY
Missing trust confidence interval:  Series([], Name: Residue, dtype: object)


  1%|▎                                        | 19/2402 [00:11<14:59,  2.65it/s]

ACY
Missing trust confidence interval:  Series([], Name: Residue, dtype: object)


  3%|█▏                                       | 71/2402 [00:40<13:42,  2.84it/s]

...

ACY
Missing trust confidence interval:  Series([], Name: Residue, dtype: object)
2219.pdb.csv 2382 Ac the attached monosaccharides does not contain a C11


100%|██████████████████████████████████████▉| 2401/2402 [20:32<00:00,  1.26it/s]

2401 2305.pdb.csv
ACY
Missing trust confidence interval:  Series([], Name: Residue, dtype: object)
ACY
Missing trust confidence interval:  Series([], Name: Residue, dtype: object)


100%|███████████████████████████████████████| 2402/2402 [20:32<00:00,  1.95it/s]

ACY
Missing trust confidence interval:  Series([], Name: Residue, dtype: object)


In [15]:
# new_pdb_f

In [16]:
# csv_f

In [17]:
# new_pdb_f_with_ac.loc[100:120]

In [18]:
# ac_hydrogen_list = ['H2']

# trust_threshold = 50

# csv_hydrogen_f['Linkage'] = csv_hydrogen_f[['Linkage']].fillna('')
# csv_hydrogen_f = csv_hydrogen_f.astype({'Linkage':str})
# if ('1.0' in csv_hydrogen_f['Linkage'].values) or ('2.0' in csv_hydrogen_f['Linkage'].values) or ('3.0' in csv_hydrogen_f['Linkage'].values) or \
#     ('4.0' in csv_hydrogen_f['Linkage'].values) or ('5.0' in csv_hydrogen_f['Linkage'].values) or ('6.0' in csv_hydrogen_f['Linkage'].values):
#     print('error in pdb lineage')

# for i in range(len(pdb_f)):
#     current_pdb_lineage = pdb_f.loc[i, :]['Lineage']
#     current_pdb_atom = pdb_f.loc[i, :]['Atom_name']
#     current_pdb_residual_name = pdb_f.loc[i, :]['Residual_name'] 

#     # stores the monosaccharide labels 
#     current_corresponding_csv_hydrogen_f = csv_hydrogen_f.loc[csv_hydrogen_f['Linkage'] == current_pdb_lineage]

#     # stores the Ac labels
#     current_corresponding_csv_hydrogen_f_ac = pd.DataFrame(columns=csv_hydrogen_f.columns)

#     if 'Ac' in current_corresponding_csv_hydrogen_f['Residue'].values:

#         current_corresponding_csv_hydrogen_f_ac = current_corresponding_csv_hydrogen_f.\
#             loc[current_corresponding_csv_hydrogen_f['Residue'] == 'Ac']

#         current_corresponding_csv_hydrogen_f = current_corresponding_csv_hydrogen_f.\
#             loc[current_corresponding_csv_hydrogen_f['Residue'] != 'Ac']

#     # current monosaccharide, assign shift,
#     if (current_pdb_atom in hydrogen_list) and (not current_corresponding_csv_hydrogen_f.empty):

#         current_shift = current_corresponding_csv_hydrogen_f[current_pdb_atom].values[0]
        
#         if current_corresponding_csv_hydrogen_f['Trust'].values[0] =='':
#             print('Missing trust confidence interval: ', current_corresponding_csv_hydrogen_f['Residue'])
#             current_trust = -1

#         elif isinstance(current_corresponding_csv_hydrogen_f['Trust'].values[0], float) and \
#             np.isnan(current_corresponding_csv_hydrogen_f['Trust'].values[0]):

#             print('Missing trust confidence interval: ', current_corresponding_csv_hydrogen_f['Residue'])
#             current_trust = -1
#         else:
#             # not monosaccharide and missing trust
#             current_trust = current_corresponding_csv_hydrogen_f['Trust'].values[0]
#             current_trust = int(current_trust.split('%')[0])

#         pdb_f.loc[i, ['shift']] = current_shift
#         pdb_f.loc[i, ['trust']] = current_trust

#     # current Ac and the pdb file has a separate three letter code
#     elif (current_pdb_atom in hydrogen_list) and (not current_corresponding_csv_hydrogen_f_ac.empty) \
#         and (current_pdb_residual_name == 'ACY'):

#         current_shift = current_corresponding_csv_hydrogen_f_ac[current_pdb_atom].values[0]
        
#         if current_corresponding_csv_hydrogen_f['Trust'].values[0] =='':
#             print('Missing trust confidence interval: ', current_corresponding_csv_hydrogen_f['Residue'])
#             current_trust = -1

#         elif isinstance(current_corresponding_csv_hydrogen_f_ac['Trust'].values[0], float) and \
#             np.isnan(current_corresponding_csv_hydrogen_f_ac['Trust'].values[0]):

#             print('Missing trust confidence interval: ', current_corresponding_csv_hydrogen_f_ac['Residue'])
#             current_trust = -1
#         else:
#             # not monosaccharide and missing trust
#             current_trust = current_corresponding_csv_hydrogen_f_ac['Trust'].values[0]
#             current_trust = int(current_trust.split('%')[0])
        
#         pdb_f.loc[i, ['shift']] = current_shift
#         pdb_f.loc[i, ['trust']] = current_trust

#     elif (current_pdb_atom in ac_hydrogen_list):
# #             print(i)
#         pass

In [19]:
[''] == ''

False

In [20]:
# for i in range(len(pdb_f)):
#     current_pdb_lineage = pdb_f.loc[i, :]['Lineage']
#     if current_pdb_lineage in csv_f['Linkage'].values:
#         current_corresponding_residue = csv_f.loc[csv_f['Linkage'] == current_pdb_lineage]['Residue'].values[0]
#     #         print(current_corresponding_residue)
#         pdb_f.loc[i, ['Residual_accurate_name']] = current_corresponding_residue

In [21]:
np.where(csv_hydrogen_f.columns == 'H1')[0][0]
csv_hydrogen_f.columns[3:].values

array(['H1', 'H2', 'H3', 'H4', 'H5', 'H6'], dtype=object)

In [22]:
# csv_f.loc[csv_f['Linkage'] == current_pdb_lineage]['Residue']

In [23]:
new_pdb_f_with_ac

,HETATM,Atom_num,Atom_name,Residual_name,Bound,Residual_num,x,y,z,Atom_type,Residual_accurate_name,Lineage,shift,trust,Ac_component,Trust
0,HETATM,1,R,NGA,A,1,0.896,9.168,17.021,X,b-D-GalpN,,-1,-1,0,NaN
1,HETATM,2,O3,NGA,A,1,-0.656,-1.103,2.529,O,b-D-GalpN,,-1,-1,0,NaN
2,HETATM,3,C3,NGA,A,1,-0.416,-0.412,3.763,C,b-D-GalpN,,77.1,80,0,NaN
3,HETATM,4,C2,NGA,A,1,-1.788,-0.157,4.458,C,b-D-GalpN,,52.1,80,0,NaN
4,HETATM,5,N2,NGA,A,1,-2.356,-1.524,4.695,N,b-D-GalpN,,-1,-1,0,NaN
5,HETATM,6,C1,NGA,A,1,-1.598,0.810,5.679,C,b-D-GalpN,,103.2,80,0,NaN
6,HETATM,7,R1,NGA,A,1,-2.912,1.531,6.119,X,b-D-GalpN,,-1,-1,0,NaN
7,HETATM,8,O5,NGA,A,1,-0.698,1.906,5.457,O,b-D-GalpN,,-1,-1,0,NaN
8,HETATM,9,C5,NGA,A,1,0.544,1.569,4.851,C,b-D-GalpN,,76.5,80,0,NaN
9,HETATM,10,C6,NGA,A,1,1.406,2.828,4.727,C,b-D-GalpN,,62.1,80,0,NaN


In [24]:
csv_hydrogen_f

,Linkage,Residue,Trust,H1,H2,H3,H4,H5,H6
0,,b-D-GalpN,81%,4.54,4.08,3.84,4.17,3.68,3.72-3.85
1,2,Ac,99%,-,2.05,,,,
2,3,a-D-Galp,89%,5.14,3.88,3.80,4.20,3.86,3.75-3.87
3,"3,3",b-D-GalpN,89%,4.82,4.18,5.15,4.31,3.79,3.76-3.88
4,"3,3,3",Ac,98%,-,2.13,,,,
5,"3,3,2",Ac,99%,-,2.05,,,,
6,"3,3,4",b-D-GlcpA,64%,4.62,3.45,3.64,3.73,3.94,-
7,"3,3,4,4",b-D-GlcpA,59%,4.51,3.53,3.76,3.86,3.88,-
8,"3,3,4,4,4",a-D-Glcp,88%,5.49,3.49,3.68,3.42,3.75,3.96-4.08


In [25]:
dict_f

{'1': '',
 '2': '3',
 '3': '3,3',
 '5': '3,3,4',
 '4': '3,3,3',
 '6': '3,3,4,4',
 '7': '3,3,4,4,4'}

In [26]:
dict_f

{'1': '',
 '2': '3',
 '3': '3,3',
 '5': '3,3,4',
 '4': '3,3,3',
 '6': '3,3,4,4',
 '7': '3,3,4,4,4'}

In [27]:
csv_f = csv_f.loc[csv_f['Residue'] != 'PDa']

In [28]:
csv_f['Linkage'] = csv_f[['Linkage']].fillna('')
csv_f = csv_f.astype({'Linkage':str})

trust_threshold = 60
ac_carbon_list = ['C11', 'C21']

for i in range(len(pdb_f)):
    current_pdb_lineage = pdb_f.loc[i, :]['Lineage']
    current_pdb_atom = pdb_f.loc[i, :]['Atom_name']
    current_pdb_residual_name = pdb_f.loc[i, :]['Residual_name']
    
    # stores the monosaccharide labels 
    current_corresponding_csv_f = csv_f.loc[csv_f['Linkage'] == current_pdb_lineage]

    # stores the Ac labels
    current_corresponding_csv_f_ac = pd.DataFrame(columns=csv_f.columns)

    if 'Ac' in current_corresponding_csv_f['Residue'].values:

        current_corresponding_csv_f_ac = current_corresponding_csv_f.\
            loc[current_corresponding_csv_f['Residue'] == 'Ac']
        
        current_corresponding_csv_f = current_corresponding_csv_f.\
            loc[current_corresponding_csv_f['Residue'] != 'Ac']
        
    # current monosaccharide, assign shift,
    if (current_pdb_atom in carbon_list) and (not current_corresponding_csv_f.empty):
        
        current_shift = current_corresponding_csv_f[current_pdb_atom].values[0]
        
        if isinstance(current_corresponding_csv_f['Trust'].values[0], float) and \
            np.isnan(current_corresponding_csv_f['Trust'].values[0]):
            
            print('Missing trust confidence interval: ', current_corresponding_csv_f['Residue'])
            current_trust = -1
        else:
            
            current_trust = current_corresponding_csv_f['Trust'].values[0]
            current_trust = int(current_trust.split('%')[0])
            
        pdb_f.loc[i, ['shift']] = current_shift
        pdb_f.loc[i, ['trust']] = current_trust
    
    # current Ac and the pdb file has a separate three letter code
    elif (current_pdb_atom in carbon_list) and (not current_corresponding_csv_f_ac.empty) \
        and (current_pdb_residual_name == 'ACY'):
        
        current_shift = current_corresponding_csv_f_ac[current_pdb_atom].values[0]
        
        if isinstance(current_corresponding_csv_f_ac['Trust'].values[0], float) and \
            np.isnan(current_corresponding_csv_f_ac['Trust'].values[0]):

            print('Missing trust confidence interval: ', current_corresponding_csv_f_ac['Residue'])
            current_trust = -1
        else:
            current_trust = current_corresponding_csv_f_ac['Trust'].values[0]
            current_trust = int(current_trust.split('%')[0])

        pdb_f.loc[i, ['shift']] = current_shift
        pdb_f.loc[i, ['trust']] = current_trust
    
    elif (current_pdb_atom in ac_carbon_list):
        print(i)

23
24
70
71


In [29]:
ac_carbon_list = ['C11', 'C21']
corres_ac_carbon_list = ['C1', 'C2']
pdb_f_lineage_list = pdb_f['Lineage'].values
for i in range(len(csv_f)):
    current_csv_lineage = csv_f.loc[i, :]['Linkage']
    current_csv_residue = csv_f.loc[i, :]['Residue']
    if current_csv_lineage not in pdb_f_lineage_list:
        current_csv_attached_lineage = current_csv_lineage[:-2]
        print(current_csv_lineage, current_csv_attached_lineage)
        
        for j in range(len(ac_carbon_list)):
            ac_c_shift = csv_f.loc[i, :][corres_ac_carbon_list[j]]
            
            ac_c_trust = csv_f.loc[i, :]['Trust']
            
            if '%' in ac_c_trust:
                ac_c_trust = int(ac_c_trust.split('%')[0])
            else:
                ac_c_trust = -1
            
#             print(ac_c_shift)
            pdb_f.loc[(pdb_f['Lineage'] == current_csv_attached_lineage) & 
                      (pdb_f['Atom_name'] == ac_carbon_list[j]), ['shift']] = ac_c_shift
    
            pdb_f.loc[(pdb_f['Lineage'] == current_csv_attached_lineage) & 
                      (pdb_f['Atom_name'] == ac_carbon_list[j]), ['trust']] = ac_c_trust

2 
3,3,2 3,3


In [30]:
pdb_f

,HETATM,Atom_num,Atom_name,Residual_name,Bound,Residual_num,x,y,z,Atom_type,Residual_accurate_name,Lineage,shift,trust
0,HETATM,1,R,NGA,A,1,0.896,9.168,17.021,X,b-D-GalpN,,-1,-1
1,HETATM,2,O3,NGA,A,1,-0.656,-1.103,2.529,O,b-D-GalpN,,-1,-1
2,HETATM,3,C3,NGA,A,1,-0.416,-0.412,3.763,C,b-D-GalpN,,77.1,80
3,HETATM,4,C2,NGA,A,1,-1.788,-0.157,4.458,C,b-D-GalpN,,52.1,80
4,HETATM,5,N2,NGA,A,1,-2.356,-1.524,4.695,N,b-D-GalpN,,-1,-1
5,HETATM,6,C1,NGA,A,1,-1.598,0.810,5.679,C,b-D-GalpN,,103.2,80
6,HETATM,7,R1,NGA,A,1,-2.912,1.531,6.119,X,b-D-GalpN,,-1,-1
7,HETATM,8,O5,NGA,A,1,-0.698,1.906,5.457,O,b-D-GalpN,,-1,-1
8,HETATM,9,C5,NGA,A,1,0.544,1.569,4.851,C,b-D-GalpN,,76.5,80
9,HETATM,10,C6,NGA,A,1,1.406,2.828,4.727,C,b-D-GalpN,,62.1,80


In [31]:
csv_hydrogen_f

,Linkage,Residue,Trust,H1,H2,H3,H4,H5,H6
0,,b-D-GalpN,81%,4.54,4.08,3.84,4.17,3.68,3.72-3.85
1,2,Ac,99%,-,2.05,,,,
2,3,a-D-Galp,89%,5.14,3.88,3.80,4.20,3.86,3.75-3.87
3,"3,3",b-D-GalpN,89%,4.82,4.18,5.15,4.31,3.79,3.76-3.88
4,"3,3,3",Ac,98%,-,2.13,,,,
5,"3,3,2",Ac,99%,-,2.05,,,,
6,"3,3,4",b-D-GlcpA,64%,4.62,3.45,3.64,3.73,3.94,-
7,"3,3,4,4",b-D-GlcpA,59%,4.51,3.53,3.76,3.86,3.88,-
8,"3,3,4,4,4",a-D-Glcp,88%,5.49,3.49,3.68,3.42,3.75,3.96-4.08


In [32]:
csv_f

,Linkage,Residue,Trust,C1,C2,C3,C4,C5,C6
0,,b-D-GalpN,80%,103.2,52.1,77.1,65.4,76.5,62.1
1,2,Ac,76%,175.7,23.6,,,,
2,3,a-D-Galp,80%,96.8,68.9,80.7,70.5,72.2,62.1
3,"3,3",b-D-GalpN,80%,104.0,52.6,73.7,72.9,75.6,61.8
4,"3,3,3",Ac,91%,174.3,21.4,,,,
5,"3,3,2",Ac,76%,175.7,23.6,,,,
6,"3,3,4",b-D-GlcpA,61%,105.5,75.4,75.2,81.3,77.0,174.5
7,"3,3,4,4",b-D-GlcpA,65%,103.5,72.8,75.8,77.9,76.9,175.0
8,"3,3,4,4,4",a-D-Glcp,81%,99.6,72.9,74.1,70.5,71.9,69.1


In [33]:
pdb_f.loc[(pdb_f['Lineage'] == current_csv_attached_lineage) & 
          (pdb_f['Atom_name'] == 'C11'), ['shift']]

,shift
70,175.7


In [34]:
current_pdb_lineage

'3,3,4,4,4'

In [35]:
current_corresponding_csv_f['Residue']

8    a-D-Glcp
Name: Residue, dtype: object

In [36]:
csv_f.loc[csv_f['Linkage'] == '4,3,8,9']

,Linkage,Residue,Trust,C1,C2,C3,C4,C5,C6


In [37]:
pdb_f

,HETATM,Atom_num,Atom_name,Residual_name,Bound,Residual_num,x,y,z,Atom_type,Residual_accurate_name,Lineage,shift,trust
0,HETATM,1,R,NGA,A,1,0.896,9.168,17.021,X,b-D-GalpN,,-1,-1
1,HETATM,2,O3,NGA,A,1,-0.656,-1.103,2.529,O,b-D-GalpN,,-1,-1
2,HETATM,3,C3,NGA,A,1,-0.416,-0.412,3.763,C,b-D-GalpN,,77.1,80
3,HETATM,4,C2,NGA,A,1,-1.788,-0.157,4.458,C,b-D-GalpN,,52.1,80
4,HETATM,5,N2,NGA,A,1,-2.356,-1.524,4.695,N,b-D-GalpN,,-1,-1
5,HETATM,6,C1,NGA,A,1,-1.598,0.810,5.679,C,b-D-GalpN,,103.2,80
6,HETATM,7,R1,NGA,A,1,-2.912,1.531,6.119,X,b-D-GalpN,,-1,-1
7,HETATM,8,O5,NGA,A,1,-0.698,1.906,5.457,O,b-D-GalpN,,-1,-1
8,HETATM,9,C5,NGA,A,1,0.544,1.569,4.851,C,b-D-GalpN,,76.5,80
9,HETATM,10,C6,NGA,A,1,1.406,2.828,4.727,C,b-D-GalpN,,62.1,80


In [38]:
csv_f

,Linkage,Residue,Trust,C1,C2,C3,C4,C5,C6
0,,b-D-GalpN,80%,103.2,52.1,77.1,65.4,76.5,62.1
1,2,Ac,76%,175.7,23.6,,,,
2,3,a-D-Galp,80%,96.8,68.9,80.7,70.5,72.2,62.1
3,"3,3",b-D-GalpN,80%,104.0,52.6,73.7,72.9,75.6,61.8
4,"3,3,3",Ac,91%,174.3,21.4,,,,
5,"3,3,2",Ac,76%,175.7,23.6,,,,
6,"3,3,4",b-D-GlcpA,61%,105.5,75.4,75.2,81.3,77.0,174.5
7,"3,3,4,4",b-D-GlcpA,65%,103.5,72.8,75.8,77.9,76.9,175.0
8,"3,3,4,4,4",a-D-Glcp,81%,99.6,72.9,74.1,70.5,71.9,69.1


In [39]:
pdb_file_list[0]

'1882.pdb.csv'

In [40]:
dict_f

{'1': '',
 '2': '3',
 '3': '3,3',
 '5': '3,3,4',
 '4': '3,3,3',
 '6': '3,3,4,4',
 '7': '3,3,4,4,4'}

In [41]:
pdb_f

,HETATM,Atom_num,Atom_name,Residual_name,Bound,Residual_num,x,y,z,Atom_type,Residual_accurate_name,Lineage,shift,trust
0,HETATM,1,R,NGA,A,1,0.896,9.168,17.021,X,b-D-GalpN,,-1,-1
1,HETATM,2,O3,NGA,A,1,-0.656,-1.103,2.529,O,b-D-GalpN,,-1,-1
2,HETATM,3,C3,NGA,A,1,-0.416,-0.412,3.763,C,b-D-GalpN,,77.1,80
3,HETATM,4,C2,NGA,A,1,-1.788,-0.157,4.458,C,b-D-GalpN,,52.1,80
4,HETATM,5,N2,NGA,A,1,-2.356,-1.524,4.695,N,b-D-GalpN,,-1,-1
5,HETATM,6,C1,NGA,A,1,-1.598,0.810,5.679,C,b-D-GalpN,,103.2,80
6,HETATM,7,R1,NGA,A,1,-2.912,1.531,6.119,X,b-D-GalpN,,-1,-1
7,HETATM,8,O5,NGA,A,1,-0.698,1.906,5.457,O,b-D-GalpN,,-1,-1
8,HETATM,9,C5,NGA,A,1,0.544,1.569,4.851,C,b-D-GalpN,,76.5,80
9,HETATM,10,C6,NGA,A,1,1.406,2.828,4.727,C,b-D-GalpN,,62.1,80


In [42]:
# pdb_f.to_csv('209_labeled.csv', index = False)

In [43]:
csv_f

,Linkage,Residue,Trust,C1,C2,C3,C4,C5,C6
0,,b-D-GalpN,80%,103.2,52.1,77.1,65.4,76.5,62.1
1,2,Ac,76%,175.7,23.6,,,,
2,3,a-D-Galp,80%,96.8,68.9,80.7,70.5,72.2,62.1
3,"3,3",b-D-GalpN,80%,104.0,52.6,73.7,72.9,75.6,61.8
4,"3,3,3",Ac,91%,174.3,21.4,,,,
5,"3,3,2",Ac,76%,175.7,23.6,,,,
6,"3,3,4",b-D-GlcpA,61%,105.5,75.4,75.2,81.3,77.0,174.5
7,"3,3,4,4",b-D-GlcpA,65%,103.5,72.8,75.8,77.9,76.9,175.0
8,"3,3,4,4,4",a-D-Glcp,81%,99.6,72.9,74.1,70.5,71.9,69.1


In [44]:
csv_hydrogen_f

,Linkage,Residue,Trust,H1,H2,H3,H4,H5,H6
0,,b-D-GalpN,81%,4.54,4.08,3.84,4.17,3.68,3.72-3.85
1,2,Ac,99%,-,2.05,,,,
2,3,a-D-Galp,89%,5.14,3.88,3.80,4.20,3.86,3.75-3.87
3,"3,3",b-D-GalpN,89%,4.82,4.18,5.15,4.31,3.79,3.76-3.88
4,"3,3,3",Ac,98%,-,2.13,,,,
5,"3,3,2",Ac,99%,-,2.05,,,,
6,"3,3,4",b-D-GlcpA,64%,4.62,3.45,3.64,3.73,3.94,-
7,"3,3,4,4",b-D-GlcpA,59%,4.51,3.53,3.76,3.86,3.88,-
8,"3,3,4,4,4",a-D-Glcp,88%,5.49,3.49,3.68,3.42,3.75,3.96-4.08


In [45]:
current_corresponding_csv_f['Residue']

8    a-D-Glcp
Name: Residue, dtype: object

In [46]:
a = csv_f.loc[csv_f['Linkage'] == '1,2,4,6']
a.loc[a['Residue'] != 'Ac']

,Linkage,Residue,Trust,C1,C2,C3,C4,C5,C6


In [47]:
csv_f

,Linkage,Residue,Trust,C1,C2,C3,C4,C5,C6
0,,b-D-GalpN,80%,103.2,52.1,77.1,65.4,76.5,62.1
1,2,Ac,76%,175.7,23.6,,,,
2,3,a-D-Galp,80%,96.8,68.9,80.7,70.5,72.2,62.1
3,"3,3",b-D-GalpN,80%,104.0,52.6,73.7,72.9,75.6,61.8
4,"3,3,3",Ac,91%,174.3,21.4,,,,
5,"3,3,2",Ac,76%,175.7,23.6,,,,
6,"3,3,4",b-D-GlcpA,61%,105.5,75.4,75.2,81.3,77.0,174.5
7,"3,3,4,4",b-D-GlcpA,65%,103.5,72.8,75.8,77.9,76.9,175.0
8,"3,3,4,4,4",a-D-Glcp,81%,99.6,72.9,74.1,70.5,71.9,69.1


In [48]:
pdb_f

,HETATM,Atom_num,Atom_name,Residual_name,Bound,Residual_num,x,y,z,Atom_type,Residual_accurate_name,Lineage,shift,trust
0,HETATM,1,R,NGA,A,1,0.896,9.168,17.021,X,b-D-GalpN,,-1,-1
1,HETATM,2,O3,NGA,A,1,-0.656,-1.103,2.529,O,b-D-GalpN,,-1,-1
2,HETATM,3,C3,NGA,A,1,-0.416,-0.412,3.763,C,b-D-GalpN,,77.1,80
3,HETATM,4,C2,NGA,A,1,-1.788,-0.157,4.458,C,b-D-GalpN,,52.1,80
4,HETATM,5,N2,NGA,A,1,-2.356,-1.524,4.695,N,b-D-GalpN,,-1,-1
5,HETATM,6,C1,NGA,A,1,-1.598,0.810,5.679,C,b-D-GalpN,,103.2,80
6,HETATM,7,R1,NGA,A,1,-2.912,1.531,6.119,X,b-D-GalpN,,-1,-1
7,HETATM,8,O5,NGA,A,1,-0.698,1.906,5.457,O,b-D-GalpN,,-1,-1
8,HETATM,9,C5,NGA,A,1,0.544,1.569,4.851,C,b-D-GalpN,,76.5,80
9,HETATM,10,C6,NGA,A,1,1.406,2.828,4.727,C,b-D-GalpN,,62.1,80


In [49]:
csv_f

,Linkage,Residue,Trust,C1,C2,C3,C4,C5,C6
0,,b-D-GalpN,80%,103.2,52.1,77.1,65.4,76.5,62.1
1,2,Ac,76%,175.7,23.6,,,,
2,3,a-D-Galp,80%,96.8,68.9,80.7,70.5,72.2,62.1
3,"3,3",b-D-GalpN,80%,104.0,52.6,73.7,72.9,75.6,61.8
4,"3,3,3",Ac,91%,174.3,21.4,,,,
5,"3,3,2",Ac,76%,175.7,23.6,,,,
6,"3,3,4",b-D-GlcpA,61%,105.5,75.4,75.2,81.3,77.0,174.5
7,"3,3,4,4",b-D-GlcpA,65%,103.5,72.8,75.8,77.9,76.9,175.0
8,"3,3,4,4,4",a-D-Glcp,81%,99.6,72.9,74.1,70.5,71.9,69.1


In [50]:
type(csv_f.loc[1, ]['C1'])

numpy.float64

In [51]:
pdb_f

,HETATM,Atom_num,Atom_name,Residual_name,Bound,Residual_num,x,y,z,Atom_type,Residual_accurate_name,Lineage,shift,trust
0,HETATM,1,R,NGA,A,1,0.896,9.168,17.021,X,b-D-GalpN,,-1,-1
1,HETATM,2,O3,NGA,A,1,-0.656,-1.103,2.529,O,b-D-GalpN,,-1,-1
2,HETATM,3,C3,NGA,A,1,-0.416,-0.412,3.763,C,b-D-GalpN,,77.1,80
3,HETATM,4,C2,NGA,A,1,-1.788,-0.157,4.458,C,b-D-GalpN,,52.1,80
4,HETATM,5,N2,NGA,A,1,-2.356,-1.524,4.695,N,b-D-GalpN,,-1,-1
5,HETATM,6,C1,NGA,A,1,-1.598,0.810,5.679,C,b-D-GalpN,,103.2,80
6,HETATM,7,R1,NGA,A,1,-2.912,1.531,6.119,X,b-D-GalpN,,-1,-1
7,HETATM,8,O5,NGA,A,1,-0.698,1.906,5.457,O,b-D-GalpN,,-1,-1
8,HETATM,9,C5,NGA,A,1,0.544,1.569,4.851,C,b-D-GalpN,,76.5,80
9,HETATM,10,C6,NGA,A,1,1.406,2.828,4.727,C,b-D-GalpN,,62.1,80


In [52]:
a = pd.read_csv('dataset/Godess_gnn/preprocess_data/Godess_carbon_name.csv')
a.loc[a['0'] == 150]

,Unnamed: 0,0,1,2
149,149,150,2065,Ac(1-6)bDGlcp(1-2)[Ac(1-6)]bDGlcp(1-17)lR17HOO...


In [53]:
# def match_pdb_csv_lineage(pdb_f, csv_f, carbon_list, ac_carbon_list = ['C11', 'C21'], trust_threshold = 50):
    
#     csv_f['Linkage'] = csv_f[['Linkage']].fillna('')
#     csv_f = csv_f.astype({'Linkage':str})
#     if ('1.0' in csv_f['Linkage'].values) or ('2.0' in csv_f['Linkage'].values) or ('3.0' in csv_f['Linkage'].values) or \
#         ('4.0' in csv_f['Linkage'].values) or ('5.0' in csv_f['Linkage'].values) or ('6.0' in csv_f['Linkage'].values):
#         print('error in pdb lineage')
    
#     for i in range(len(pdb_f)):
#         current_pdb_lineage = pdb_f.loc[i, :]['Lineage']
#         current_pdb_atom = pdb_f.loc[i, :]['Atom_name']
#         current_pdb_residual_name = pdb_f.loc[i, :]['Residual_name'] 

#         # stores the monosaccharide labels 
#         current_corresponding_csv_f = csv_f.loc[csv_f['Linkage'] == current_pdb_lineage]

#         # stores the Ac labels
#         current_corresponding_csv_f_ac = pd.DataFrame(columns=csv_f.columns)

#         if 'Ac' in current_corresponding_csv_f['Residue'].values:

#             current_corresponding_csv_f_ac = current_corresponding_csv_f.\
#                 loc[current_corresponding_csv_f['Residue'] == 'Ac']

#             current_corresponding_csv_f = current_corresponding_csv_f.\
#                 loc[current_corresponding_csv_f['Residue'] != 'Ac']

#         # current monosaccharide, assign shift,
#         if (current_pdb_atom in carbon_list) and (not current_corresponding_csv_f.empty):

#             current_shift = current_corresponding_csv_f[current_pdb_atom].values[0]

#             if isinstance(current_corresponding_csv_f['Trust'].values[0], float) and \
#                 np.isnan(current_corresponding_csv_f['Trust'].values[0]):

#                 print('Missing trust confidence interval: ', current_corresponding_csv_f['Residue'])
#                 current_trust = -1
#             else:

#                 current_trust = current_corresponding_csv_f['Trust'].values[0]
#                 current_trust = int(current_trust.split('%')[0])

#             pdb_f.loc[i, ['shift']] = current_shift
#             pdb_f.loc[i, ['trust']] = current_trust

#         # current Ac and the pdb file has a separate three letter code
#         elif (current_pdb_atom in carbon_list) and (not current_corresponding_csv_f_ac.empty) \
#             and (current_pdb_residual_name == 'ACY'):

#             current_shift = current_corresponding_csv_f_ac[current_pdb_atom].values[0]

#             if isinstance(current_corresponding_csv_f_ac['Trust'].values[0], float) and \
#                 np.isnan(current_corresponding_csv_f_ac['Trust'].values[0]):

#                 print('Missing trust confidence interval: ', current_corresponding_csv_f_ac['Residue'])
#                 current_trust = -1
#             else:
#                 current_trust = current_corresponding_csv_f_ac['Trust'].values[0]
#                 current_trust = int(current_trust.split('%')[0])

#             pdb_f.loc[i, ['shift']] = current_shift
#             pdb_f.loc[i, ['trust']] = current_trust

#         elif (current_pdb_atom in ac_carbon_list):
# #             print(i)
#             pass
        
#     return pdb_f